In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sys, re
import numpy as np
from random import random
from operator import add
import pymongo
# from pymongo import MongoClient
import os
from os import listdir

In [14]:
#initialize database
'''client = MongoClient()
db = client.bookdb
posts = db.posts'''
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["posts"]
mycol = mydb["posts"]
mydict = {"name": "John", "address": "Highway 37"}
x = mycol.insert_one(mydict)

In [17]:
print(myclient.list_database_names())

['admin', 'config', 'local', 'mydatabase', 'posts']


In [18]:
#load in content
#AI: do stemming before running this part. should improve results.
documents = []
docsDir = "C://Users//krgau//OneDrive//Desktop//data//t1//"

for book in os.listdir(docsDir):
    #print(book)
    if not book.startswith('.'): #pass hidden files such as .DS_STORE
        book = str(book) #file name
        print(book)
        with open("C:/Users/krgau/OneDrive/Desktop/data/t1/" + book, 'rb') as f:
            content = f.read() #.splitlines()
            content = str(content, errors='replace')
            documents.append(content) 
            #print(content)
#print(documents)

12370-8.txt
12370.txt
12372-8.txt
12372.txt
12373-8.txt
12373.txt
12374-8.txt
12374.txt


In [19]:
#print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
tfidf = TfidfVectorizer(max_df=0.9,
                        ngram_range=(1, 1),
                        stop_words='english',
                        strip_accents='unicode', analyzer = 'word')

In [21]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='unicode',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [22]:
#documents = documents.toLocalIterator()
tfidf_matrix =  tfidf.fit_transform(documents)
feature_names = tfidf.get_feature_names() 

In [23]:
tfidf_matrix

<8x19982 sparse matrix of type '<class 'numpy.float64'>'
	with 58336 stored elements in Compressed Sparse Row format>

In [24]:
title_author_store = []
for book in os.listdir(docsDir):
	title = None
	author = None
	#read file
	if not book.startswith('.'):    #pass hidden files such as .DS_STORE
		book = str(book)
		with open("C:/Users/krgau/OneDrive/Desktop/data/t/" + book, 'r') as f:
			content = f.read().splitlines()
			#print(content)
		#find title and author
		for i in range(80):
			if "Title: " in content[i]:
				title = content[i][7:]
			if "Author: " in content[i]:
				author = content[i][8:]
		title_author_tuple = (title, author)
		title_author_store.append(title_author_tuple) 


In [25]:
title_author_store

[('Bagh O Bahar, Or Tales of the Four Darweshes', 'Mir Amman of Dihli'),
 ('Bagh O Bahar, Or Tales of the Four Darweshes', 'Mir Amman of Dihli'),
 ('Atlantic Monthly, Vol. 1, No. 6, April, 1858', 'Various'),
 ('Atlantic Monthly, Vol. 1, No. 6, April, 1858', 'Various'),
 ('Atlantic Monthly, Vol. 1, No. 5, March, 1858', 'Various'),
 ('Atlantic Monthly, Vol. 1, No. 5, March, 1858', 'Various'),
 ('Atlantic Monthly, Vol. 1, No. 7, May, 1858', 'Various'),
 ('Atlantic Monthly, Vol. 1, No. 7, May, 1858', 'Various')]

In [26]:
#populate dict
#verify: sorting doesn't seem necessary here. can probably save time by killing that
database = {}
for i in range(tfidf_matrix.shape[0]):
	doc = tfidf_matrix[i].toarray()[0]
	#currently using 1-grams, but potential for use of n-grams with phrases here
	phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]
	#get top results 
	sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
	#create dict of {word: score} for fast lookups
	local_word_dict = {}
	for pair in sorted_phrase_scores:
		term = feature_names[pair[0]].encode('ascii', 'ignore').decode('ascii')    #encoding issues keep breaking the mongo queries
		local_word_dict[term] = pair[1]
	database[(title_author_store[i])] = local_word_dict 

In [30]:
for i in database.keys():
    print(i)
    print(database[i])
    print("\n\n\n\n\n")

('Bagh O Bahar, Or Tales of the Four Darweshes', 'Mir Amman of Dihli')
{'princess': 0.44263596905313646, '_khwaja_': 0.19365819335049347, 'eunuch': 0.1757268791513737, '_darweshes_': 0.13269172507348626, '_wazir_': 0.13269172507348626, '_mubarak_': 0.12551919939383838, '_hatim_': 0.11117414803454255, 'favour': 0.1075878851947186, 's_': 0.10691690073747256, 'slaves': 0.10691690073747256, 'female': 0.10050188669322421, '_khil': 0.09682909667524674, '_wazir': 0.09682909667524674, 'bakht_': 0.09682909667524674, '_azad': 0.09324283383542278, '_dilli_': 0.08607030815577488, '_jinns_': 0.08607030815577488, 'seraglio': 0.08607030815577488, 'idol': 0.08553352058997805, 'persia': 0.08553352058997805, 'persian': 0.08553352058997805, 'nurse': 0.08411312117262508, 'honour': 0.08248404531595092, 'nobles': 0.08139979468318556, 'adventures': 0.0791185065457297, 'wretch': 0.0791185065457297, '_hindustan_': 0.07889778247612697, 'governor': 0.07698016853098025, '_maliki': 0.07531151963630302, 'sadik_': 0

In [53]:

for i in database.keys():
	title = i[0]
	author = i[1]
	try:
		posts = {"title_id_0011": str(title), "author_id_0011": str(author)}
# 		print(posts)        
		words = database[i]
# 		print(words.items())    
# 		posts =dict(posts.items() + words.items()) #faster to query if no nested dicts
		x=posts.update(words)
# 		print(posts)
# 		post_id = mycol.insert_one(posts).inserted_id
		post_id = mycol.insert_one(posts).inserted_id
	except:
		print (str(title) + ", " + str(author) + " failed")
#         x = mycol.insert_one(mydict)

In [55]:
#now populate the mongo collection
for i in database.keys():   #for doc in dictionary
	title = i[0].encode('ascii', 'ignore').decode('ascii') #trying to get around the encoding issues
	author = i[1].encode('ascii', 'ignore').decode('ascii') 
	try:
		posts = {"title_id_0011": str(title), "author_id_0011": str(author)}
		words = database[i]
# 		posts = dict(post.items() + words.items()) #combine words with title/author dict. faster to query if no nested dicts
		x=posts.update(words)
		post_id = mycol.insert_one(posts).inserted_id #add record to db
	except:
		print (str(title) + ", " + str(author) + " failed") #encoding issues. find a way to fix these w/o needing the exception handling.

In [66]:
for x in mycol.find():
    print(x)

{'_id': ObjectId('5e62a5d84f04922444fcacb2'), 'title_id_0011': 'Bagh O Bahar, Or Tales of the Four Darweshes', 'author_id_0011': 'Mir Amman of Dihli', 'princess': 0.44263596905313646, '_khwaja_': 0.19365819335049347, 'eunuch': 0.1757268791513737, '_darweshes_': 0.13269172507348626, '_wazir_': 0.13269172507348626, '_mubarak_': 0.12551919939383838, '_hatim_': 0.11117414803454255, 'favour': 0.1075878851947186, 's_': 0.10691690073747256, 'slaves': 0.10691690073747256, 'female': 0.10050188669322421, '_khil': 0.09682909667524674, '_wazir': 0.09682909667524674, 'bakht_': 0.09682909667524674, '_azad': 0.09324283383542278, '_dilli_': 0.08607030815577488, '_jinns_': 0.08607030815577488, 'seraglio': 0.08607030815577488, 'idol': 0.08553352058997805, 'persia': 0.08553352058997805, 'persian': 0.08553352058997805, 'nurse': 0.08411312117262508, 'honour': 0.08248404531595092, 'nobles': 0.08139979468318556, 'adventures': 0.0791185065457297, 'wretch': 0.0791185065457297, '_hindustan_': 0.0788977824761269

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

